In [3]:
import json
import logging
import os
import traceback
from pathlib import Path
from typing import Dict, List

import pandas as pd
import xlwings as xw
from pydantic import ValidationError
from sixgill.definitions import *
from sixgill.pipesim import Model, Units

from core import NetworkSimulation, PipSimInput

logger = logging.getLogger("Hydraulics")

with open("inputs.json", "r") as f:
    config = json.load(f)

try:
    config = PipSimInput(**config)
    FOLDER_DIRECTORY = config.FOLDER_DIRECTORY
    MODEL_FILENAME = config.MODEL_FILENAME
    EXCEL_FILE = config.EXCEL_FILE
    PIPSIM_INPUT_SHEET = config.PIPSIM_INPUT_SHEET
    CONDITIONS_SHEET = config.CONDITIONS_SHEET
    SOURCE_NAME = config.SOURCE_NAME
    PUMP_NAME = config.PUMP_NAME
except (ValidationError, KeyError) as e:
    logging.error(e)

INFO:Input Validation:PIPSIM Input Validation Successful
INFO:Input Validation:PIPSIM Input Validation Successful


In [4]:
case_list = ["NorthHeader_Max"]
conditions_list = ["S-HP-EO", "S-LP-EO", "W-HP-EO", "W-LP-EO", "S-LP-LO", "W-LP-LO"]

In [5]:
# filename "NorthHeader_Max_W-LP-LO_BAB HP.pips"
filelist = []
for case_name in case_list:
    for condition_name in conditions_list:
        filename = f"{case_name}_{condition_name}_BAB HP.pips"
        filelist.append(filename)
filelist

['NorthHeader_Max_S-HP-EO_BAB HP.pips',
 'NorthHeader_Max_S-LP-EO_BAB HP.pips',
 'NorthHeader_Max_W-HP-EO_BAB HP.pips',
 'NorthHeader_Max_W-LP-EO_BAB HP.pips',
 'NorthHeader_Max_S-LP-LO_BAB HP.pips',
 'NorthHeader_Max_W-LP-LO_BAB HP.pips']

In [6]:
for MODEL_FILENAME in filelist:
    ns = NetworkSimulation(FOLDER_DIRECTORY, MODEL_FILENAME, EXCEL_FILE)
    ns.initialize_excel_handler(PIPSIM_INPUT_SHEET, CONDITIONS_SHEET)
    ns.run_existing_model(source_name=SOURCE_NAME, pump_name=PUMP_NAME)

INFO:NetworkSimulation:------------Network Simulation Object Created----------------
Model Path: c:\Users\IDM252577\Desktop\Python Projects\PIPSIM\NorthHeader_Max_S-HP-EO_BAB HP.pips

INFO:NetworkSimulation:Model prepared
 Case:NorthHeader_Max
 Condition: S-HP-EO
INFO:manta.server.manager:Starting PIPESIM server on thread ID: 18752
INFO:manta.server.manager:Waiting for PIPESIM server to start.
INFO:manta.server.manager:Python toolkit license is checked out successfully
INFO:NetworkSimulation:Getting boundary conditions.....
INFO:NetworkSimulation:Set Global Conditions
INFO:sixgill.core.run_or_start_simulation:Simulation is running.
INFO:sixgill.core.run_or_start_simulation:Simulation is running.
INFO:sixgill.core.run_or_start_simulation:Simulation is running.
INFO:sixgill.core.run_or_start_simulation:Simulation is running.
INFO:sixgill.core.run_or_start_simulation:Simulation is running.
INFO:sixgill.core.run_or_start_simulation:Simulation is running.
INFO:sixgill.core.run_or_start_simu

In [7]:
import pandas as pd

# The path to your Excel file
excel_file_path = "Node Results.xlsx"

# Initialize an empty list to store DataFrames
dfs = []

# Use Pandas to read the Excel file and load each sheet
xls = pd.ExcelFile(excel_file_path)

for sheet_name in xls.sheet_names:
    # Read specific range of each sheet, with header in first row (Python index 0) and no index column
    df = pd.read_excel(
        xls,
        sheet_name=sheet_name,
        usecols="B:G",
        header=0,
        index_col=None,
        skiprows=1,
        nrows=3,
    )
    # Add a new column to the DataFrame with the sheet name
    df["Sheet Name"] = sheet_name
    # Append the DataFrame to the list
    dfs.append(df)

# Combine all DataFrames in the list into a single DataFrame
combined_df = pd.concat(dfs, ignore_index=True)

# Show the combined DataFrame
print(combined_df)

        Node  Type    Pressure  Temperature  VolumeFlowrateWaterStockTank  \
0   WIF16_17  Sink  244.323281    18.167403                   4350.900016   
1     IWS003  Sink  268.750624    16.907376                    606.590002   
2   WIF16_17  Sink  195.010377    18.102261                   4350.900016   
3     IWS003  Sink  219.456753    16.823166                    606.590002   
4   WIF16_17  Sink  257.321068    18.176155                   4350.900016   
..       ...   ...         ...          ...                           ...   
59    IWS003  Sink  219.634193    47.725067                    562.070002   
60  WIF16_17  Sink  259.786294    49.702423                   4319.600016   
61    IWS003  Sink  281.922915    47.818582                    562.070002   
62  WIF16_17  Sink  200.964125    49.599780                   4319.600016   
63    IWS003  Sink  223.133428    47.704645                    562.070002   

    Min/Max                  Sheet Name  
0   Minimum      Overall_Max_W-LP

In [8]:
import xlwings as xw

wb = xw.Book("Node Results backup.xlsx")
ws = wb.sheets[0]
ws.range("B1").value = combined_df